# Fast Agent - Build a SQL Agent fast!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages

In [ ]:
'''from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")'''

In [ ]:
# Quick demo: connect to the Banking demo DB and run example queries
from langchain_community.utilities import SQLDatabase

# Path to the generated BankingDemo.db (created by tools/create_banking_db.py)
bank_db = SQLDatabase.from_uri("sqlite:///BankingDemo.db")

'''queries = [
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;",
    "SELECT c.customer_id, c.first_name || ' ' || c.last_name AS name, SUM(a.balance) AS total_balance FROM customer c JOIN account a ON c.customer_id = a.customer_id GROUP BY c.customer_id ORDER BY total_balance DESC LIMIT 5;",
    "SELECT a.account_id, a.account_number, a.balance FROM account a ORDER BY a.balance DESC LIMIT 5;",
    "SELECT t.transaction_id, t.account_id, t.txn_date, t.txn_type, t.amount FROM transactions t ORDER BY t.txn_date DESC LIMIT 10;"
]

for q in queries:
    print("--- QUERY:\n", q)
    try:
        res = bank_db.run(q)
        print(res)
    except Exception as e:
        print('Error running query:', e)
    print()'''

Define the runtime context to provide the agent and tools with access to the database.

In [ ]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

<b>⚠️ Security Note:</b> This demo does not include a filter on LLM-generated commands. In production, you would want to limit the scope of LLM-generated commands. ⚠️   
This tool will connect to the database. Note the use of `get_runtime` to access the graph **runtime context**.

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Add a system prompt to define your agents behavior.

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again. If error pertains to table or column names not available, explore available tables and columns to find the needed information.
- Prefer explicit column lists; avoid SELECT *.
- If requested data is not available in the database and multiple retries fail, respond with "Requested information is not available in the database."
"""

Create your agent! Add a model, tools, a prompt, and the runtime access, and go!  You can choose many agents from our [integrations](https://docs.langchain.com/oss/python/integrations/providers) list.

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

'''agent = create_agent(
    model="groq:llama-3.3-70b-versatile",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)'''


Here's a display of the agent ReAct Loop.

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

Run some queries. Notice:
- The agent does not have the database schema and will need to discover it independently.
- The agent may make mistakes! By returning error messages, the agent can self-correct its queries.
- Notice you invoke the agent with `agent.stream`.
    - This command and the `pretty_print` display the **messages** that communicate information between the model and the tools.
- Notice the agent doesn't remember the schema between invocations... More on this later!

In [ ]:
question = "Which table has the most number of entries?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=bank_db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "what are your branch timings?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=bank_db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Please list all of the tables, their columns and number of entries in them"

for step in agent.stream(
    {"messages": question},
    context={"db": bank_db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**Create your own query here!**  Add some questions of your own.

In [ ]:
question = "List top 3 branches with highest number of accounts"

for step in agent.stream(
    {"messages": question},
    context={"db": bank_db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Let's try this Studio